In [ ]:
from numpy import expand_dims
from numpy import mean
from numpy import ones
from numpy.random import randn
from numpy.random import randint
import tensorflow as tf

from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras import backend
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.constraints import Constraint
from matplotlib import pyplot
 
# clip model weights to a given hypercube

 
# calculate wasserstein loss


In [ ]:
class ClipConstraint(Constraint):
	# set clip value when initialized
	def __init__(self, clip_value):
		self.clip_value = clip_value
 
	# clip model weights to hypercube
	def __call__(self, weights):
		return backend.clip(weights, -self.clip_value, self.clip_value)
 
	# get the config
	def get_config(self):
		return {'clip_value': self.clip_value}

In [ ]:
def wasserstein_loss(y_true, y_pred):
	return backend.mean(y_true * y_pred)


In [ ]:
def define_critic(in_shape=(28,28,1)):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# weight constraint
	const = ClipConstraint(0.01)
	# define model
	model = Sequential()
	# downsample to 14x14
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, kernel_constraint=const, input_shape=in_shape))
	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 7x7
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, kernel_constraint=const))
	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	# scoring, linear activation
	model.add(Flatten())
	model.add(Dense(1))
	# compile model
	opt = RMSprop(lr=0.00005)
	model.compile(loss=wasserstein_loss, optimizer=opt)
	return model

# define the standalone generator model
def define_generator(latent_dim):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# define model
	model = Sequential()
	# foundation for 7x7 image
	n_nodes = 128 * 7 * 7
	model.add(Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((7, 7, 128)))
	# upsample to 14x14
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 28x28
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	# output 28x28x1
	model.add(Conv2D(1, (7,7), activation='tanh', padding='same', kernel_initializer=init))
	return model

# define the combined generator and critic model, for updating the generator
def define_gan(generator, critic):
	# make weights in the critic not trainable
	for layer in critic.layers:
		if not isinstance(layer, BatchNormalization):
			layer.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the critic
	model.add(critic)
	# compile model
	opt = RMSprop(lr=0.00005)
	model.compile(loss=wasserstein_loss, optimizer=opt)
	return model

# load images
def load_real_samples():
	# load dataset
	(trainX, trainy), (_, _) = load_data()
	# select all of the examples for a given class
	selected_ix = trainy == 7
	X = trainX[selected_ix]
	# expand to 3d, e.g. add channels
	X = expand_dims(X, axis=-1)
	# convert from ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return X

# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# select images
	X = dataset[ix]
	# generate class labels, -1 for 'real'
	y = -ones((n_samples, 1))
	return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = generator.predict(x_input)
	# create class labels with 1.0 for 'fake'
	y = ones((n_samples, 1))
	return X, y

# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
	# prepare fake examples
	X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
	# scale from [-1,1] to [0,1]
	X = (X + 1) / 2.0
	# plot images
	for i in range(10 * 10):
		# define subplot
		pyplot.subplot(10, 10, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
	# save plot to file
	filename1 = 'generated_plot_%04d.png' % (step+1)
	pyplot.savefig(filename1)
	pyplot.close()
	# save the generator model
	filename2 = 'model_%04d.h5' % (step+1)
	g_model.save(filename2)
	print('>Saved: %s and %s' % (filename1, filename2))

# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist):
	# plot history
	pyplot.plot(d1_hist, label='crit_real')
	pyplot.plot(d2_hist, label='crit_fake')
	pyplot.plot(g_hist, label='gen')
	pyplot.legend()
	pyplot.savefig('plot_line_plot_loss.png')
	pyplot.close()

# train the generator and critic
def train(g_model, c_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=64, n_critic=5):
	# calculate the number of batches per training epoch
	bat_per_epo = int(dataset.shape[0] / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# calculate the size of half a batch of samples
	half_batch = int(n_batch / 2)
	# lists for keeping track of loss
	c1_hist, c2_hist, g_hist = list(), list(), list()
	# manually enumerate epochs
	for i in range(n_steps):
		# update the critic more than the generator
		c1_tmp, c2_tmp = list(), list()
		for _ in range(n_critic):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update critic model weights
			c_loss1 = c_model.train_on_batch(X_real, y_real)
			c1_tmp.append(c_loss1)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update critic model weights
			c_loss2 = c_model.train_on_batch(X_fake, y_fake)
			c2_tmp.append(c_loss2)
		# store critic loss
		c1_hist.append(mean(c1_tmp))
		c2_hist.append(mean(c2_tmp))
		# prepare points in latent space as input for the generator
		X_gan = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = -ones((n_batch, 1))
		# update the generator via the critic's error
		g_loss = gan_model.train_on_batch(X_gan, y_gan)
		g_hist.append(g_loss)
		# summarize loss on this batch
		print('>%d, c1=%.3f, c2=%.3f g=%.3f' % (i+1, c1_hist[-1], c2_hist[-1], g_loss))
		# evaluate the model performance every 'epoch'
		if (i+1) % bat_per_epo == 0:
			summarize_performance(i, g_model, latent_dim)
	# line plots of loss
	plot_history(c1_hist, c2_hist, g_hist)

In [ ]:
latent_dim = 50
# create the critic
critic = define_critic()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, critic)
# load image data
dataset = load_real_samples()
print(dataset.shape)
# train model
train(generator, critic, gan_model, dataset, latent_dim)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


11501568/11490434 [==============================] - 0s 0us/step
(6265, 28, 28, 1)
>1, c1=-2.375, c2=0.042 g=-0.646
>2, c1=-6.617, c2=0.122 g=-2.157
>3, c1=-9.605, c2=0.182 g=-3.103
>4, c1=-11.926, c2=0.251 g=-3.911
>5, c1=-14.558, c2=0.304 g=-4.998
>6, c1=-16.003, c2=0.386 g=-6.174
>7, c1=-17.631, c2=0.457 g=-7.079
>8, c1=-18.846, c2=0.527 g=-8.649
>9, c1=-20.386, c2=0.622 g=-10.074
>10, c1=-21.648, c2=0.702 g=-11.126
>11, c1=-22.239, c2=0.772 g=-12.077
>12, c1=-23.491, c2=0.870 g=-13.441
>13, c1=-24.527, c2=0.971 g=-13.978
>14, c1=-25.545, c2=1.052 g=-14.880
>15, c1=-26.264, c2=1.172 g=-15.446
>16, c1=-26.920, c2=1.232 g=-16.030
>17, c1=-27.437, c2=1.330 g=-16.698
>18, c1=-27.766, c2=1.439 g=-17.136
>19, c1=-29.410, c2=1.476 g=-17.840
>20, c1=-29.253, c2=1.481 g=-18.464
>21, c1=-30.441, c2=1.600 g=-18.719
>22, c1=-30.966, c2=1.594 g=-19.119
>23, c1=-31.253, c2=1.663 g=-19.511
>24, c1=-31.894, c2=1.595 g=-20.450
>25, c1=-32.384, c2=1.624 g=-19.815
>26, c1=-32.788, c2=1.590 g=-20.517
>

>Saved: generated_plot_0097.png and model_0097.h5
>98, c1=-75.059, c2=-63.336 g=-74.367
>99, c1=-76.576, c2=-63.731 g=-74.940
>100, c1=-77.661, c2=-64.858 g=-75.759
>101, c1=-77.880, c2=-65.315 g=-75.862
>102, c1=-79.036, c2=-66.461 g=-77.032
>103, c1=-78.765, c2=-66.887 g=-77.987
>104, c1=-80.026, c2=-67.769 g=-78.926
>105, c1=-81.065, c2=-68.248 g=-79.712
>106, c1=-81.759, c2=-69.338 g=-80.642
>107, c1=-81.363, c2=-69.720 g=-81.415
>108, c1=-83.264, c2=-70.927 g=-81.897
>109, c1=-83.506, c2=-71.491 g=-83.354
>110, c1=-83.867, c2=-72.374 g=-83.661
>111, c1=-85.137, c2=-72.900 g=-84.466
>112, c1=-85.286, c2=-74.006 g=-85.456
>113, c1=-86.365, c2=-74.533 g=-86.149
>114, c1=-87.403, c2=-75.580 g=-86.964
>115, c1=-87.712, c2=-76.348 g=-87.616
>116, c1=-87.944, c2=-76.977 g=-88.747
>117, c1=-88.964, c2=-77.864 g=-89.561
>118, c1=-89.955, c2=-78.312 g=-90.653
>119, c1=-90.884, c2=-79.434 g=-91.852
>120, c1=-91.547, c2=-80.010 g=-92.081
>121, c1=-91.359, c2=-81.115 g=-92.713
>122, c1=-92.233

>Saved: generated_plot_0194.png and model_0194.h5
>195, c1=-147.110, c2=-139.894 g=-156.524
>196, c1=-148.873, c2=-140.701 g=-157.525
>197, c1=-149.653, c2=-141.513 g=-158.457
>198, c1=-150.832, c2=-142.450 g=-159.286
>199, c1=-151.408, c2=-142.361 g=-159.888
>200, c1=-151.999, c2=-144.174 g=-160.648
>201, c1=-152.489, c2=-145.168 g=-161.584
>202, c1=-154.853, c2=-145.224 g=-162.384
>203, c1=-155.363, c2=-146.976 g=-163.605
>204, c1=-156.089, c2=-147.132 g=-164.693
>205, c1=-156.484, c2=-147.636 g=-164.667
>206, c1=-157.438, c2=-149.645 g=-166.016
>207, c1=-158.001, c2=-149.572 g=-165.392
>208, c1=-158.407, c2=-151.521 g=-167.377
>209, c1=-159.985, c2=-152.112 g=-168.659
>210, c1=-160.956, c2=-152.377 g=-169.292
>211, c1=-161.516, c2=-153.766 g=-170.228
>212, c1=-162.101, c2=-154.478 g=-171.380
>213, c1=-162.737, c2=-154.463 g=-171.565
>214, c1=-163.378, c2=-156.222 g=-173.086
>215, c1=-165.070, c2=-156.221 g=-173.679
>216, c1=-165.251, c2=-157.975 g=-174.902
>217, c1=-166.251, c2=-157

>Saved: generated_plot_0291.png and model_0291.h5
>292, c1=-231.173, c2=-226.002 g=-242.905
>293, c1=-232.904, c2=-227.873 g=-244.563
>294, c1=-234.728, c2=-226.175 g=-245.749
>295, c1=-233.902, c2=-225.824 g=-244.742
>296, c1=-233.663, c2=-230.232 g=-246.865
>297, c1=-236.983, c2=-230.194 g=-248.266
>298, c1=-237.068, c2=-229.307 g=-248.734
>299, c1=-237.099, c2=-232.113 g=-249.978
>300, c1=-238.614, c2=-228.361 g=-249.023
>301, c1=-239.628, c2=-233.886 g=-251.513
>302, c1=-240.369, c2=-231.760 g=-251.571
>303, c1=-241.398, c2=-235.862 g=-253.025
>304, c1=-240.086, c2=-237.068 g=-254.178
>305, c1=-243.060, c2=-231.500 g=-254.482
>306, c1=-241.829, c2=-235.591 g=-252.125
>307, c1=-245.312, c2=-241.552 g=-257.210
>308, c1=-247.914, c2=-240.008 g=-258.411
>309, c1=-244.063, c2=-240.065 g=-257.413
>310, c1=-248.569, c2=-243.987 g=-260.176
>311, c1=-249.346, c2=-243.430 g=-260.948
>312, c1=-250.383, c2=-243.081 g=-261.991
>313, c1=-251.047, c2=-244.169 g=-262.877
>314, c1=-250.114, c2=-243

KeyboardInterrupt: ignored